In [ ]:
import os
from qdrant_client.http.models import Batch
from qdrant_client.http import models
from qdrant_client import QdrantClient
import json
from sentence_transformers import SentenceTransformer

from dotenv import load_dotenv
load_dotenv("../")

In [ ]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
annotations = json.load(open("../annotations.json"))


In [ ]:

vector_size = encoder.get_sentence_embedding_dimension()

def get_embedding(text: str) -> list:
    # embedding = openai.embeddings.create(
    #     input=[text],
    #     model=embedding_model,
    # ).data[0].embedding
    embedding = encoder.encode(text).tolist()
    
    return embedding


In [ ]:
for i, annotation in enumerate(annotations):
    print(annotations[annotation])
    break

In [ ]:
qc = QdrantClient("localhost", port=6333)

In [ ]:
collections = qc.get_collections()
collection_name = "annotations"
if collection_name not in [collection.name for collection in collections.collections]:
    first_collection = qc.recreate_collection(
        collection_name=collection_name,
        vectors_config = models.VectorParams(size=vector_size, distance=models.Distance.COSINE)
    )

qc.get_collections().collections[0].name

In [ ]:
def annotation_to_vector(annotation: dict) -> models.Vector:
    combined_text = annotation["name"] + " " + ", ".join([a["name"] for a in annotation['tags']])
    print(f"Combined text: {combined_text}")
    return get_embedding(combined_text)

for i, annotation in enumerate(annotations):
    vector = annotation_to_vector(annotations[annotation])
    print(vector)
    break

In [ ]:
len(annotations)

In [ ]:
for id, annotation in enumerate(annotations):
    # check
    # query_filter = models.Filter(must=[models.HasIdCondition(has_id=[annotation])])
    # res = qc.scroll(
        # collection_name=collection_name,
        # scroll_filter=query_filter,
    # )
    # print(f"Res: for {annotations[annotation]} {res}")
    # if len(res[0]) > 0:
        # continue

    if annotations[annotation].keys() != {"uid", "name", "tags", "viewerUrl"}:
        continue
    qc.upsert(
        collection_name=collection_name,
        points=Batch(
            ids=[id],
            vectors=[annotation_to_vector(annotations[annotation])],
            payloads=[
                {
                    "name": annotations[annotation]["name"],
                    "uid": annotations[annotation]["uid"],
                    "tags": annotations[annotation]["tags"],
                }
            ],
        ),
    )

In [ ]:
resume = "person"

res = qc.search(
    collection_name=collection_name,
    query_vector=get_embedding(resume),
)

res